## Labeling

In [1]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"ilooeey","key":"f7610ad4d2bfa7f0fd18e59b1817ac37"}'}

In [2]:
# !pip install Sastrawi

In [3]:
!pip install -q kaggle
!mkdir -p ~/.kaggle

!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets list


In [4]:
import pandas as pd
url = 'https://www.kaggle.com/datasets/iqbalmaulana/indonesian-news-dataset'
!kaggle datasets download -d iqbalmaulana/indonesian-news-dataset
!unzip -q indonesian-news-dataset.zip -d indonesian_news


Dataset URL: https://www.kaggle.com/datasets/iqbalmaulana/indonesian-news-dataset
License(s): CC-BY-NC-SA-4.0
indonesian-news-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
replace indonesian_news/data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


In [5]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Load data
df = pd.read_csv('indonesian_news/data.csv')[:3000]
import pandas as pd
import re



# List kata kunci isu sensitif
sensitive_keywords = [
    # Politik
    'jokowi', 'prabowo', 'dpr', 'presiden', 'menteri', 'politik', 'partai', 'pemilu',
    # Agama
    'agama', 'islam', 'kristen', 'haji', 'umroh', 'ulama', 'syariah',
    # Kekerasan / kriminalitas
    'bom', 'pembunuhan', 'pemerkosaan', 'penembakan', 'teroris', 'ledakan',
    # Sosial
    'demo', 'kerusuhan', 'rasis', 'ham', 'diskriminasi', 'konflik',
    # Hoaks
    'hoaks', 'provokatif', 'fitnah'
]

# Fungsi bersihkan teks
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Fungsi cek apakah teks mengandung kata sensitif
def is_sensitive(text, keywords):
    return any(keyword in text for keyword in keywords)

# Terapkan labeling
df['label_sensitive'] = df.apply(
    lambda row: int(
        is_sensitive(
            clean_text(row['title']) + ' ' + clean_text(row['content']),
            sensitive_keywords
        )
    ),
    axis=1
)
import ast

df['embedding'] = df['embedding'].apply(ast.literal_eval)


In [6]:
df['clean_text'] = df['content'].apply(clean_text)


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

X = df['embedding'].tolist()
y = df['label_sensitive']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.71      0.65      0.68       160
           1       0.88      0.90      0.89       440

    accuracy                           0.83       600
   macro avg       0.79      0.78      0.78       600
weighted avg       0.83      0.83      0.83       600



In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pandas as pd


# 1. TF-IDF vektorisasi
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X = vectorizer.fit_transform(df['clean_text'])

kmeans = KMeans(n_clusters=2, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

sensitive_keywords = ['korupsi', 'agama', 'ras', 'politik', 'seksual', 'provokasi', 'kontroversi', 'HAM']

def sensitivity_score(text):
    return sum(1 for word in sensitive_keywords if word in text.lower())

cluster_scores = df.groupby('cluster')['clean_text'].apply(lambda texts: sum(sensitivity_score(t) for t in texts))

sensitive_cluster_label = cluster_scores.idxmax()

#labeling
df['is_sensitive'] = df['cluster'].apply(lambda x: 1 if x == sensitive_cluster_label else 0)
print("\nDistribusi isu sensitif:")
print(df['is_sensitive'].value_counts())

for label in df['cluster'].unique():
    print(f"\nContoh dari cluster {label} (Sensitif: {label == sensitive_cluster_label}):")
    print(df[df['cluster'] == label]['clean_text'].sample(5, random_state=1))

# 9. Simpan hasil
df.to_csv('hasil_deteksi_isu_sensitif.csv', index=False)



Distribusi isu sensitif:
is_sensitive
1    2913
0      87
Name: count, dtype: int64

Contoh dari cluster 0 (Sensitif: True):
1885    info nasional upaya mendorong tumbuhnya wiraus...
634     kepala ppatk ivanyustiavandanamenegaskan pihak...
2027    pt jasa marga persero bakal menegur petugas ya...
811     puluhan warga memadati area rekonstruksi kasus...
618     bareskrim polri tengah menyelidiki laporan pt ...
Name: clean_text, dtype: object

Contoh dari cluster 1 (Sensitif: False):
1027    nan
1437    nan
843     nan
1119    nan
963     nan
Name: clean_text, dtype: object


In [9]:
dp = pd.read_csv('/content/hasil_deteksi_isu_sensitif.csv')
dp

,id,source,title,image,url,content,date,embedding,created_at,updated_at,summary,label_sensitive,clean_text,cluster,is_sensitive
0,83,tempo,"Depo Plumpang Terbakar, Anggota DPR Minta Pert...",https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698528/depo-pl...,"TEMPO.CO, Jakarta - Anggota Komisi VII DPR RI ...",2023-03-04 06:18:13+00,"[-0.01590039, -0.034130897, 0.005732614, -0.01...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Anggota Komisi VII DPR RI Rofik Hananto menyay...,1,tempoco jakarta anggota komisi vii dpr ri rofi...,0,1
1,84,tempo,Jokowi Perintahkan Wapres Ma'ruf Amin Tinjau L...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698522/jokowi-...,"TEMPO.CO, Jakarta - Presiden Joko Widodo atau ...",2023-03-04 06:04:38+00,"[-0.017608976, -0.021786924, 0.01547983, -0.00...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Presiden Joko Widodo telah memerintahkan Wakil...,1,tempoco jakarta presiden joko widodo atau joko...,0,1
2,85,tempo,HNW Mendukung Jamaah Umroh First Travel Dapatk...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698527/hnw-men...,INFO NASIONAL - Wakil Ketua MPR RI Dr. H. M. H...,2023-03-04 06:18:04+00,"[0.00841488, -0.023665192, 0.006762431, -0.013...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Wakil Ketua MPR RI Dr. H. M. Hidayat Nur Wahid...,1,info nasional wakil ketua mpr ri dr h m hidaya...,0,1
3,86,tempo,Tim Dokkes Polri Telah Terima 14 Kantong Jenaz...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698540/tim-dok...,"TEMPO.CO, Jakarta - Tim Kedokteran dan Kesehat...",2023-03-04 06:44:10+00,"[-0.012671886, -0.0039057182, 0.019575326, -0....",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Tim Kedokteran dan Kesehatan (Dokkes) Polri te...,0,tempoco jakarta tim kedokteran dan kesehatan d...,0,1
4,87,tempo,Bamsoet Ajak Komunitas Otomotif Kembangkan Per...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698536/bamsoet...,INFO NASIONAL - Ketua MPR RI sekaligus Ketua U...,2023-03-04 06:38:57+00,"[-0.015486176, -0.0125719, -0.0122843925, -0.0...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Ketua MPR RI Bambang Soesatyo telah diangkat s...,1,info nasional ketua mpr ri sekaligus ketua umu...,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,7949,tempo,Penutupan Silicon Valley Bank Diyakini Tidak B...,https://statik.tempo.co/data/2023/03/14/id_118...,https://bisnis.tempo.co/read/1702502/penutupan...,"TEMPO.CO, Jakarta -Menteri Koordinator Bidang ...",2023-03-14 09:32:39+00,"[-0.0054657464, -0.0345414, 0.017403042, -0.01...",2023-03-14 10:00:22.521853,2023-03-14 10:00:22.521853,Menteri Koordinator Bidang Kemaritiman dan Inv...,1,tempoco jakarta menteri koordinator bidang kem...,0,1
2996,7950,tempo,AHY Khawatir Penundaan Pemilu Bikin Chaos: Apa...,https://statik.tempo.co/data/2023/03/02/id_118...,https://nasional.tempo.co/read/1702498/ahy-kha...,"TEMPO.CO, Jakarta - Ketua Umum Partai Demokrat...",2023-03-14 09:30:00+00,"[0.0013776253, -0.021576328, 0.004430391, -0.0...",2023-03-14 10:00:22.521853,2023-03-14 10:00:22.521853,AHY mengkhawatirkan putusan pengadilan atas pe...,1,tempoco jakarta ketua umum partai demokrat agu...,0,1
2997,7951,tempo,KPK akan Verifikasi Laporan IPW terhadap Edwar...,https://statik.tempo.co/data/2022/11/09/id_115...,https://nasional.tempo.co/read/1702512/kpk-aka...,"TEMPO.CO, Jakarta - Kepala Bagian Pemberitaan ...",2023-03-14 09:48:23+00,"[0.010492585, -0.022113826, -0.000757497, -0.0...",2023-03-14 10:00:22.521853,2023-03-14 10:00:22.521853,KPK akan menelaah laporan Ketua IPW Sugeng Teg...,1,tempoco jakarta kepala bagian pemberitaan kpk ...,0,1
2998,26798,okezone,TGB Jelaskan 3 Dimensi Ramadhan yang Penting D...,https://img.okezone.com/dynamic/content/2023/0...,https://nasional.okezone.com/read/2023/03/22/3...,JAKARTA Jelang penentuan awal bulan Ramadh

## MLP